In [80]:
import pandas as pd

In [81]:
df = pd.read_csv('../data/kaggle_bias/train.csv')

In [82]:
test = pd.read_csv('../data/kaggle_bias/test_public_expanded.csv')

Create new binary target column for modelling

In [83]:
df['binary_target'] = (df['target'] > 0.5).astype(int)
test['binary_target'] = (test['toxicity'] > 0.5).astype(int)

In [84]:
df[['id', 'comment_text', 'target', 'binary_target']]

,id,comment_text,target,binary_target
0,59848,"This is so cool. It's like, 'would you want yo...",0.000000,0
1,59849,Thank you!! This would make my life a lot less...,0.000000,0
2,59852,This is such an urgent design problem; kudos t...,0.000000,0
3,59855,Is this something I'll be able to install on m...,0.000000,0
4,59856,haha you guys are a bunch of losers.,0.893617,1
...,...,...,...,...
1804869,6333967,"Maybe the tax on ""things"" would be collected w...",0.000000,0
1804870,6333969,What do you call people who STILL think the di...,0.000000,0
1804871,6333982,"thank you ,,,right or wrong,,, i am following ...",0.000000,0
1804872,6334009,Anyone who is quoted as having the following e...,0.621212,1


Remove nulls

In [85]:
df.dropna(subset = ['comment_text'], inplace=True)

In [86]:
len(df)

1804871

In [87]:
df['binary_target'].sum()

106438

In [88]:
(df['binary_target'].sum() / len(df) ) * 100

5.897263571745571

5.96% of our data is the target class in train data

In [89]:
(test['binary_target'].sum() / len(test) ) * 100

5.804562268803946

5.80% of our data is the target class in test data

In [90]:
df = df.sample(frac=0.2, random_state=11)

In [91]:
X = df[ 'comment_text']
y = df['target']
y_bin = df['binary_target']

In [92]:
X_test = test['comment_text']
y_test = test['toxicity']
y_bin_test = test['binary_target']

In [93]:
X.isnull().sum()

0

In [94]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import string

In [95]:
stop_words = stopwords.words('english')

In [96]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [38]:
def custom_tokenizer(document):
    
    #clean up
    document = document.replace('\n', '')
    
    # remove punctuation
    for punct in string.punctuation:
        document=document.replace(punct,'')

    # tokenize - split on whitespace
    tokenized_document = document.split(' ')
    # only include word if it is NOT = ''
    tokenized_document = [word for word in tokenized_document if word != '']

    # remove stopwords before stemming
    tokenized_document = [word for word in tokenized_document if word not in stop_words]

    #stem words and return list of stemmed tokens
    stemmed_tokens_list = []
    for i in tokenized_document:
        token = PorterStemmer().stem(i)
        stemmed_tokens_list.append(token)
    return stemmed_tokens_list

    return tokenized_document

In [39]:
count_vectorizer = CountVectorizer(tokenizer=custom_tokenizer, min_df=10, max_features=3000, ngram_range=(1,3))

X_fit = count_vectorizer.fit(X)
X_transformed = count_vectorizer.transform(X)  

C:\Users\arman\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [97]:
X_test_transformed = count_vectorizer.transform(X_test)  

In [98]:
tokens_df = pd.DataFrame(
        data = X_transformed.todense(), 
        columns = count_vectorizer.get_feature_names_out(), index=X.index)

In [99]:
tokens_df

,1,10,10 year,100,1000,11,12,13,14,15,...,younger,your,your right,youth,youv,zero,zone,zuma,–,—
1519250,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
394286,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
303890,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
845158,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1500302,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
734652,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
847935,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
213946,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24193,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [100]:
test_tokens_df = pd.DataFrame(
        data = X_test_transformed.todense(), 
        columns = count_vectorizer.get_feature_names_out(), index=X_test.index)

In [101]:
test_tokens_df

,1,10,10 year,100,1000,11,12,13,14,15,...,younger,your,your right,youth,youv,zero,zone,zuma,–,—
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97315,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
97316,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
97317,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
97318,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [102]:
from sklearn.linear_model import LogisticRegression, LinearRegression
import numpy as np
import matplotlib.pyplot as plt

### Linear Regression

In [45]:
linear_model=LinearRegression()
linear_model.fit(tokens_df,y)

LinearRegression()

In [46]:
print(f"Train set accuracy {linear_model.score(X_transformed,y):.2f}%")

Train set accuracy 0.36%


C:\Users\arman\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [47]:
print(f"Test set accuracy {linear_model.score(X_test_transformed,y_test):.2f}%")

Test set accuracy 0.35%


C:\Users\arman\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [51]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [49]:
y_pred = linear_model.predict(X_test_transformed)

C:\Users\arman\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [50]:
mean_absolute_error(y_test, y_pred)

0.10997959652508191

In [52]:
mean_squared_error(y_test, y_pred)

0.024907439432313456

1st iteration, base linear regression no optimization at any step - 3000 max features, ngram(1,3), min 10:
- 36% train
- 35% test

2nd iteration, base linear regression, removed '\n' in tokenizer - 500 max features, ngram(1,3), min 10:
- 15% train
- 15% test
- mean absolute error 0.128

3rd iteration, base linear regression, removed '\n' in tokenizer - 3000 max features, ngram(1,3), min 10:
- 36% train
- 35% test
- mean absolute error 0.109
- mean square error 0.024

In [ ]:
# run log reg on binarized target and pull out top contributing tokens to modify vectorizer for linear regression

### Logistic Regression

In [64]:
logistic_model=LogisticRegression(max_iter=1000) #C=1 by default
logistic_model.fit(tokens_df,y_bin)

LogisticRegression(max_iter=1000)

In [103]:
print(f"Train set accuracy {logistic_model.score(X_transformed,y_bin):.2f}%")

Train set accuracy 0.96%


C:\Users\arman\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


In [104]:
print(f"Test set accuracy {logistic_model.score(X_test_transformed,y_bin_test):.2f}%")

Test set accuracy 0.95%


C:\Users\arman\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


In [ ]:
1st iteration, base linear regression no optimization at any step - 3000 max features, ngram(1,3), min 10:
- 36% train
- 35% test